In [739]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import classification_report
from sklearn import preprocessing

import time
from datetime import datetime

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

%matplotlib inline

from catboost import CatBoostClassifier

In [740]:
df = pd.read_csv("./content/train_enriched.csv")

In [741]:
abc_le = preprocessing.LabelEncoder()
df["ABC"] = abc_le.fit_transform(df["ABC"])

In [742]:
df['control_year'] = pd.to_datetime(df['month_id']).dt.strftime("%Y").astype(int)
df['control_month'] = pd.to_datetime(df['month_id']).dt.strftime("%m").astype(int)
df['control_timestamp'] = df['month_id'].apply(lambda d: time.mktime(datetime.strptime(d, "%m/%d/%Y").timetuple()))

df['cart_day'] = pd.to_datetime(df['carts_created_at']).dt.strftime("%d").astype(int)
df['cart_month'] = pd.to_datetime(df['carts_created_at']).dt.strftime("%m").astype(int)
df['cart_year'] = pd.to_datetime(df['carts_created_at']).dt.strftime("%Y").astype(int)
df['cart_timestamp'] = df['carts_created_at'].apply(lambda d: time.mktime(datetime.strptime(d, "%m/%d/%Y").timetuple()))

In [743]:
df['promo']=df['promo'].apply(lambda promo: promo =='+').astype(int)

In [744]:
ct_le = preprocessing.LabelEncoder()
df["communication_type"] = ct_le.fit_transform(df["communication_type"])

In [745]:
os_le = preprocessing.LabelEncoder()
df["os"] = os_le.fit_transform(df["os"])

browser_le = preprocessing.LabelEncoder()
df["browser"] = browser_le.fit_transform(df["browser"])

platform_le = preprocessing.LabelEncoder()
df["platform"] = platform_le.fit_transform(df["platform"])

program_family_price_type_le = preprocessing.LabelEncoder()
df["program_family_price_type"] = program_family_price_type_le.fit_transform(df["program_family_price_type"])

program_type_le = preprocessing.LabelEncoder()
df["program_type"] = program_type_le.fit_transform(df["program_type"])

program_starting_soon_le = preprocessing.LabelEncoder()
df["program_starting_soon"] = program_starting_soon_le.fit_transform(df["program_starting_soon"])

In [746]:
df['current_program_starts_on_day'] = pd.to_datetime(df['current_program_starts_on']).dt.strftime("%d")
df['current_program_starts_on_month'] = pd.to_datetime(df['current_program_starts_on']).dt.strftime("%m")
df['current_program_starts_on_year'] = pd.to_datetime(df['current_program_starts_on']).dt.strftime("%Y")

In [747]:
def to_timestamp(d):
    if not isinstance(d, float):
        d = time.mktime(datetime.strptime(d, "%Y-%m-%d").timetuple())
    return d

In [748]:
df['current_program_starts_on_timestamp'] = df['current_program_starts_on'].apply(lambda d: to_timestamp(d))

In [749]:
df['current_program_starts_on_timestamp']

0         1.602450e+09
1                  NaN
2         1.659560e+09
3         1.664140e+09
4                  NaN
              ...     
199995             NaN
199996    1.650834e+09
199997    1.660165e+09
199998             NaN
199999    1.660252e+09
Name: current_program_starts_on_timestamp, Length: 200000, dtype: float64

In [750]:
def transform_duration(row):
    program_duration = str(row.program_duration)
    splt =  program_duration.split(' ')
    unit = splt[1][0:3]
    number = int(splt[0])
    if unit =='мес':
        number=number * 4
    row['program_duration'] = number
    return row


In [751]:
df[['program_duration']]=df[['program_duration']].fillna('0 недель')
df=df.apply(lambda row: transform_duration(row), axis=1)

In [752]:
def replace_country(row):
    country=row['country']
    if country in ['Россия', '<span>Россия</span>', 'Росссия','РФ']:
         row['country']= "Россия"
    return row

In [753]:
# df = df.apply(lambda row: replace_country(row), axis=1)

In [754]:
# country_le = preprocessing.LabelEncoder()
# df["country"] = country_le.fit_transform(df["country"])

In [ ]:
# df.info()

In [ ]:
# df.dtypes[df.dtypes == "object"].values

In [ ]:
# mass_object = df.dtypes[df.dtypes == "object"].index.values

In [ ]:
# mass_object

In [ ]:
# df = df.drop(['available_program_starts_on', 'carts_created_at', 'city',
#        'current_program_starts_on', 'month_id',], axis = 1)

In [ ]:
x_feats=["program_id",
"price",
 "cart_timestamp",
"control_month",
"student_id",
"age_indicator",
"feedback_avg_d1",
"feedback_avg_d3",
"feedback_avg_d5",
"feedback_avg_d4",
"feedback_avg_d2",
"control_year",
"support_feedback_avg",
"ABC",
"promo" , 
       'bought_d1', 'bought_d2', 'bought_d3', 'bought_d4',
       'bought_d5', 
          'bought_avg_duration',
         "m_was_conversations","p_was_conversations",
  
       "program_family_main_direction_id",
       "program_type",
#          "program_starting_soon",
       "current_program_starts_on_timestamp", 
          "time_diff"
     ]
feats=x_feats+[   "target" ]

In [ ]:
feats

In [ ]:
# df.info()

In [ ]:
# df=df[feats]

In [ ]:
# df.info()

In [ ]:
 df = df.fillna(0)

# imp = IterativeImputer(initial_strategy='median')
# df = pd.DataFrame(data=imp.fit_transform(df), columns=df.columns)

#df = df.dropna()

# cols =[
#     "age_indicator",
# "feedback_avg_d1",
# "feedback_avg_d3",
# "feedback_avg_d5",
# "feedback_avg_d4",
# "feedback_avg_d2",
# "support_feedback_avg",
# "bought_avg_duration",
# "m_was_conversations",
# "p_was_conversations",
# "current_program_starts_on_timestamp",
# ]

# df=df[~df[cols].isna().all(axis=1)]

In [ ]:
df["time_diff"] = df["current_program_starts_on_timestamp"] - df["cart_timestamp"]

In [ ]:
df[df.age_indicator<15]

In [ ]:
# def age_transform(row):
#     age = row.age_indicator
#     if age<10:
#         row['age_indicator']= 20
#     return row

In [ ]:
# df = df.apply(lambda row: age_transform(row), axis=1)

In [ ]:
df=df[feats]

In [ ]:
# df['current_program_starts_on_day'] = df['current_program_starts_on_day'].astype(int)
# df['current_program_starts_on_month'] = df['current_program_starts_on_month'].astype(int)
# df['current_program_starts_on_year'] = df['current_program_starts_on_year'].astype(int)

In [ ]:
# df.info()

In [ ]:
# df = df.fillna(0)

In [ ]:
df

In [ ]:
X =  df.drop(["target"], axis = 1)
y = df[["target"]]

In [ ]:
X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
clf = RandomForestClassifier(random_state=0, max_depth=50, n_estimators=500)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
0.2* recall_score(y_test, pred, average='macro') + 0.8* precision_score(y_test, pred, average='macro')

In [ ]:
clf.classes_

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
importance_df = pd.DataFrame(clf.feature_importances_, columns=["value"],
                             index=clf.feature_names_in_).sort_values(by="value", ascending=False)
importance_df

#Validation

In [ ]:
df_test = pd.read_csv("./content/test_enriched.csv")

In [ ]:
df_test["ABC"] = abc_le.transform(df_test["ABC"])

In [ ]:
df_test['control_year'] = pd.to_datetime(df_test['month_id']).dt.strftime("%Y").astype(int)
df_test['control_month'] = pd.to_datetime(df_test['month_id']).dt.strftime("%m").astype(int)
df_test['control_timestamp'] = df_test['month_id'].apply(lambda d: time.mktime(datetime.strptime(d, "%m/%d/%Y").timetuple()))

df_test['cart_day'] = pd.to_datetime(df_test['carts_created_at']).dt.strftime("%d").astype(int)
df_test['cart_month'] = pd.to_datetime(df_test['carts_created_at']).dt.strftime("%m").astype(int)
df_test['cart_year'] = pd.to_datetime(df_test['carts_created_at']).dt.strftime("%Y").astype(int)
df_test['cart_timestamp'] = df_test['carts_created_at'].apply(lambda d: time.mktime(datetime.strptime(d, "%m/%d/%Y").timetuple()))

In [ ]:
df_test['promo']=df['promo'].apply(lambda promo: promo =='+').astype(int)

In [ ]:
df_test["communication_type"] = ct_le.transform(df_test["communication_type"])

In [ ]:
df_test["os"] = os_le.transform(df_test["os"])
df_test["platform"] = platform_le.transform(df_test["platform"])

In [ ]:
df_test["program_type"] = program_type_le.transform(df_test["program_type"])
df_test["program_starting_soon"] = program_starting_soon_le.transform(df_test["program_starting_soon"])

In [ ]:
program_family_price_type_le_dict = dict(zip(program_family_price_type_le.classes_, program_family_price_type_le.transform(program_family_price_type_le.classes_)))
df_test['program_family_price_type'] = df_test['program_family_price_type'].apply(lambda program_family_price_type: program_family_price_type_le_dict.get(program_family_price_type, -1))

In [ ]:
df_test['current_program_starts_on_day'] = pd.to_datetime(df_test['current_program_starts_on']).dt.strftime("%d")
df_test['current_program_starts_on_month'] = pd.to_datetime(df_test['current_program_starts_on']).dt.strftime("%m")
df_test['current_program_starts_on_year'] = pd.to_datetime(df_test['current_program_starts_on']).dt.strftime("%Y")

In [ ]:
df_test['current_program_starts_on_timestamp'] = df_test['current_program_starts_on'].apply(lambda d: to_timestamp(d))

In [ ]:
df_test[['program_duration']]=df_test[['program_duration']].fillna('0 недель')
df_test=df_test.apply(lambda row: transform_duration(row), axis=1)

In [ ]:
browser_le_dict = dict(zip(browser_le.classes_, browser_le.transform(browser_le.classes_)))
df_test['browser'] = df_test['browser'].apply(lambda browser: browser_le_dict.get(browser, -1))

In [ ]:
# df_test = df_test.apply(lambda row: age_transform(row), axis=1)

In [ ]:
# df_test = df_test.apply(lambda row: replace_country(row), axis=1)

In [ ]:
# df_test["country"] = country_le.transform(df_test["country"])

In [ ]:
df_test.info()

In [ ]:
mass_object_v = mass_object[mass_object!="target"]

In [ ]:
# X_val = df_test.drop(mass_object_v, axis = 1)

In [ ]:
df_test = df_test.fillna(0)

In [ ]:
df_test["time_diff"] = df_test["current_program_starts_on_timestamp"] - df_test["cart_timestamp"]

In [ ]:
X_val=df_test[x_feats]

In [ ]:
pred_test = clf.predict(X_val)

In [ ]:
df_test["target"] = pred_test

In [ ]:
df_test=df_test[["id", "target"]]

In [ ]:
df_test.to_csv("./content/pred.csv", index=False)

In [830]:
# X_val = df_test.drop(mass_object_v, axis = 1)

In [831]:
df_test = df_test.fillna(0)

In [832]:
df_test["time_diff"] = df_test["current_program_starts_on_timestamp"] - df_test["cart_timestamp"]

In [833]:
X_val=df_test[x_feats]

In [834]:
pred_test = clf.predict(X_val)

In [835]:
df_test["target"] = pred_test

In [836]:
df_test=df_test[["id", "target"]]

In [837]:
df_test.to_csv("./content/pred.csv", index=False)